In [1]:
!pip install torchvision torch

In [2]:
!pip install datasets

In [6]:
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

In [7]:
# !pip install --upgrade urllib3
# !pip install --upgrade accelerate
# !pip install --upgrade transformers

In [8]:
import os

import datasets
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split

# from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
# from transformers import VisionEncoderDecoderModel , ViTFeatureExtractor
# from transformers import AutoTokenizer ,  GPT2Config , default_data_collator
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import clip


if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
class config: 
    ENCODER = "google/vit-base-patch16-224" # ruclip-vit-base-patch32-384
    DECODER = "gpt2" # rugpt3large_based_on_gpt2
    TRAIN_BATCH_SIZE = 8
    VAL_BATCH_SIZE = 8
    VAL_EPOCHS = 1
    LR = 5e-5
    SEED = 42
    MAX_LEN = 128
    SUMMARY_LEN = 20
    WEIGHT_DECAY = 0.01
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)
    TRAIN_PCT = 0.95
    NUM_WORKERS = mp.cpu_count()
    EPOCHS = 1
    IMG_SIZE = (224, 224)
    LABEL_MASK = -100
    TOP_K = 1000
    TOP_P = 0.95
    PREFIX_LENGTH = 50

# Dataset

## Download

In [ ]:
'''import requests
import zipfile
import os

images_url = "http://images.cocodataset.org/zips/train2017.zip"
annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
 
download_folder = "coco_data"

# Создаем папку, если ее нет
os.makedirs(download_folder, exist_ok=True)

# Скачивание и распаковка изображений
images_zip_path = os.path.join(download_folder, "train2017.zip")
annotations_zip_path = os.path.join(download_folder, "annotations_trainval2017.zip")

# Скачивание изображений
with requests.get(images_url, stream=True) as r:
    with open(images_zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# Скачивание аннотаций
with requests.get(annotations_url, stream=True) as r:
    with open(annotations_zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# Распаковка архивов
with zipfile.ZipFile(images_zip_path, 'r') as zip_ref:
    zip_ref.extractall(download_folder)

with zipfile.ZipFile(annotations_zip_path, 'r') as zip_ref:
    zip_ref.extractall(download_folder)

# Удаление загруженных zip-архивов (если нужно)
os.remove(images_zip_path)
os.remove(annotations_zip_path)'''

In [23]:
import json

# Путь к файлу с аннотациями
json_file_path = 'coco_data/annotations/captions_train2017.json'

# Путь к текстовому файлу, в который будем записывать данные
output_txt_path = 'coco_data/annotations/captions_train2017.txt'

# Чтение JSON-файла
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
    print(len(data['images']))
    print(data['images'][0]['file_name'])
    print(data['annotations'][0]['caption'])

# Открытие текстового файла для записи
# with open(output_txt_path, 'w') as output_txt_file:
#     # Обход каждого элемента в JSON-файле
#     for annotation in data['annotations']:
#         # Получение пути к файлу изображения и аннотации
#         image_path = annotation['file_name']
#         caption = annotation['caption']
        
#         # Запись в текстовый файл с разделением табуляцией
#         output_txt_file.write(f"{image_path}\t{caption}\n")

# print(f"Annotations extracted and written to {output_txt_path}")

118287
000000391895.jpg
A bicycle replica with a clock as the front wheel.


### Create DataFrame

In [79]:
import pandas as pd
import json

# Путь к файлу с аннотациями
json_file_path = 'coco_data/annotations/captions_train2017.json'

# Чтение JSON-файла
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Создание списка для данных
annotations_data = {'image_path': [],
                   'caption': []}

# Обход каждой аннотации в JSON-файле
for index in range(0, 118287):
    annotations_data['image_path'].append(data['images'][index]['file_name'])
    annotations_data['caption'].append(data['annotations'][index]['caption'])


df_train = pd.DataFrame(annotations_data)

print(df_train.shape)
df_train.head()

(118287, 2)


,image_path,caption
0,000000391895.jpg,A bicycle replica with a clock as the front wh...
1,000000522418.jpg,A room with blue walls and a white sink and door.
2,000000184613.jpg,A car that seems to be parked illegally behind...
3,000000318219.jpg,A large passenger airplane flying through the ...
4,000000554625.jpg,There is a GOL plane taking off in a partly cl...


In [78]:
import pandas as pd
import json

# Путь к файлу с аннотациями
json_file_path = 'coco_data/annotations/captions_val2017.json'

# Чтение JSON-файла
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Создание списка для данных
annotations_data = {'image_path': [],
                   'caption': []}

# Обход каждой аннотации в JSON-файле
for index in range(0, 5000):
    annotations_data['image_path'].append(data['images'][index]['file_name'])
    annotations_data['caption'].append(data['annotations'][index]['caption'])


df_val = pd.DataFrame(annotations_data)

print(df_val.shape)
df_val.head()

(5000, 2)


,image_path,caption
0,000000397133.jpg,A black Honda motorcycle parked in front of a ...
1,000000037777.jpg,A Honda motorcycle parked in a grass driveway
2,000000252219.jpg,An office cubicle with four different types of...
3,000000087038.jpg,A small closed toilet in a cramped space.
4,000000174482.jpg,Two women waiting at a bench next to a street.


## ImgDataset

In [147]:
train_df , val_df = train_test_split(df_train, test_size = 0.2)

In [46]:
feature_extractor = ViTFeatureExtractor.from_pretrained(config.ENCODER)
tokenizer = GPT2LMHeadModel.from_pretrained(config.DECODER)
tokenizer.pad_token = tokenizer.unk_token

(…)24/resolve/main/preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/Users/kekgerman/opt/anaconda3/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [204]:
transforms = transforms.Compose(
    [
        transforms.Resize(config.IMG_SIZE),
        transforms.ToTensor(),
        # transforms.Normalize(
        # mean=0.5,
        # std=0.5,
        # ),
   ]
)

In [205]:
# tokenizer(df_train['caption'].iloc[5], padding='max_length', max_length=50).input_ids

In [206]:
# print(df_train['caption'].iloc[0])
# print(df_train['image_path'].iloc[0])
# transforms(Image.open(df_train['image_path'].iloc[4]).convert("RGB"))

In [225]:
class ImgDataset(Dataset):
    def __init__(self, df, root_dir, tokenizer, feature_extractor, transform = None):
        self.df = df
        self.transform = transform
        self.root_dir = root_dir
        self.tokenizer= tokenizer
        self.feature_extractor = feature_extractor
        self.max_length = 50
        
    def __len__(self,):
        return len(self.df)
    
    def __getitem__(self, idx):
        caption = self.df['caption'].iloc[idx]
        image = self.df['image_path'].iloc[idx]
        img_path = os.path.join(self.root_dir, image)
        img = Image.open(img_path)
        
        if self.transform is not None:
            img = self.transform(img)
        
        # print(img)
        pixel_values = self.feature_extractor(img, return_tensors="pt").pixel_values
        captions = self.tokenizer(caption,
                                  padding='max_length',
                                  max_length=self.max_length).input_ids
        
        tokens = torch.tensor(captions)
        prefix = pixel_values
        return tokens, prefix

In [226]:
train_dataset = ImgDataset(train_df, root_dir = "coco_data/train2017", tokenizer=tokenizer, feature_extractor = feature_extractor, transform = transforms)
val_dataset = ImgDataset(val_df, root_dir = "coco_data/train2017", tokenizer=tokenizer, feature_extractor = feature_extractor, transform  = transforms)

In [227]:
train_dataset[0]

(tensor([  464,  9283,  2137, 26728,   465,  7365,   379,   281, 13885,  9283,
            13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),
 tensor([[[[-0.9942, -0.9942, -0.9942,  ..., -0.9938, -0.9937, -0.9935],
           [-0.9942, -0.9942, -0.9942,  ..., -0.9937, -0.9936, -0.9934],
           [-0.9942, -0.9942, -0.9942,  ..., -0.9937, -0.9936, -0.9934],
           ...,
           [-0.9988, -0.9988, -0.9988,  ..., -0.9930, -0.9932, -0.9934],
           [-0.9988, -0.9988, -0.9988,  ..., -0.9930, -0.9931, -0.9933],
           [-0.9988, -0.9988, -0.9988,  ..., -0.9930, -0.9930, -0.9932]],
 
          [[-0.9942, -0.9942, -0.9942,  ..., -0.9938, -0.9938, -0.9938],
           [-0.9942, -0.9942, -0.9942,  ..., -0.9938, -0.9937, -0.9937],


# Main

## model

In [2]:
# !pip install pip install --upgrade transformers

  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2


In [13]:
from typing import Tuple, Optional
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel
from torch.cuda.amp import autocast


class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
    
    @autocast()  
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)
    

class ClipCaptionModel(nn.Module):
    def __init__(self, backbone, prefix_length: int, prefix_size: int = 24):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained(backbone)
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                self.gpt_embedding_size * prefix_length))

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    @autocast() 
    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):

        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        print("Prefix projections shape:", prefix_projections.shape)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

  
class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
Failed to import transformers.integrations.peft because of the following error (look up to see its traceback):
cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (/Users/kekgerman/opt/anaconda3/lib/python3.9/site-packages/urllib3/util/ssl_.py)

## train

In [ ]:
def get_caption(prefix, model, device, tokenizer, prompt=''):
    prefix = prefix.to(device)
    with torch.no_grad():

        prefix_embed = model.clip_project(prefix).reshape(len(prefix), config.PREFIX_LENGHT, -1)

        answers = []

        for x in range(len(prefix_embed)):

            start = time.time()

            cur_prefix_embed = prefix_embed[x].unsqueeze(0).to('cpu')
            
            print('first', time.time()-start)
            
            if prompt:
                generated_text_prefix = generate2(model, tokenizer, prompt=prompt, embed=cur_prefix_embed)
            else:
                generated_text_prefix = generate2(model, tokenizer, embed=cur_prefix_embed)
            
            print('second', time.time()-start)
        
            answers.append(generated_text_prefix.replace('\n',' ').replace('<|endoftext|',''))

    return [x[len(prompt):].strip() for x in answers]

# def get_ans(model, clip_emb, prompt, device, tokenizer):
#     output = get_caption(clip_emb, model, device, tokenizer, prompt=prompt)
#     return output

In [4]:
def train(train_loader, model, optimizer, scheduler, device, epoch):
    loss_avg = AverageMeter()
    
    model = model.to(device)
    model.train()

    progress = tqdm(total=len(train_loader))
    for idx, (tokens, prefix) in enumerate(train_loader):
        model.zero_grad()
        tokens, prefix = tokens.to(device), prefix.to(device, dtype=torch.float32)
        
        outputs = model(tokens, prefix)
        logits = outputs.logits[:, config.PREFIX_LENGHT-1: -1]

        loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

        segments = 2

        # out = checkpoint_sequential(modules, segments, input_var)

        loss.backward()    
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        clipping_value = 0.5 # arbitrary value of your choosing
        #torch.nn.utils.clip_grad_norm(model.parameters(), clipping_value)

        loss_avg.update(loss.item(), len(mask))
        progress.set_description(f"loss: {loss_avg.avg:.5f}")
        progress.update()
        

        # del tokens
        # del mask
        # del prefix
        torch.clear_autocast_cache()
        torch.cuda.empty_cache()
    progress.close()

    return model


def valid(model, valid_loader, device, gt):
    loss_avg = AverageMeter()
    model.eval()
    progress = tqdm(enumerate(valid_loader), total=len(valid_loader))
    tokenizer = GPT2Tokenizer.from_pretrained(CFG.backbone)
    

    all_answers = []
    for idx, (tokens, prefix) in progress:
        tokens, prefix = tokens.to(device), prefix.to(device, dtype=torch.float32)
        answer = get_caption(prefix, model, device, tokenizer, prompt='Caption: ')
        all_answers.append(answer)
    score = bleu_metric(gt, np.concatenate(all_answers))
    return score
    

import nltk
def bleu_metric(ground_truth, prediction):
    scores = []
    for gt, pred in zip(ground_truth, prediction):
        if type(pred)==str and type(gt)==str:
            score = nltk.translate.bleu_score.sentence_bleu([gt.lower().split()], pred.lower().replace('<|endoftext|>','').split(), weights = (0.5, 0.5))
        scores+=[score]
    return np.array(scores).mean()*100

## main

In [5]:
# import bitsandbytes as bnb
import gc
import io
import os
import random
import numpy as np
from tqdm import tqdm
import pandas as pd
import cv2
import sys

import torchvision
import transformers
import torch
from torch.nn import functional as nnf
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.cuda.amp import autocast

from transformers.optimization import Adafactor, AdafactorSchedule
from torch.utils.checkpoint import checkpoint_sequential
import warnings
warnings.simplefilter('ignore')

import time

In [6]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [7]:
def main():
    loss_avg = AverageMeter()
    
    train_loader = DataLoader(train_dataset, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=config.VAL_BATCH_SIZE, shuffle=False)

    model = ClipCaptionModel(prefix_length = config.PREFIX_LENGTH, backbone = config.DECODER)

    # model.load_state_dict(torch.load('/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/v2_1.pt', map_location='cpu'))
    model = model.to(device)
   
    #model = freeze(model)

    model.train()
    optimizer = AdamW(model.parameters(), lr=0.33, betas=(0.9, 0.995))
    #optimizer = bnb.optim.Adam8bit(model.parameters(), lr=0.001, betas=(0.9, 0.995))
    #optimizer = SM3(model.parameters(),lr=args.lr)
    #Adafactor(model.parameters(),scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=3, num_training_steps=config.EPOCHS * len(train_loader))
    #AdafactorSchedule(optimizer)#num_training_steps=epochs * len(train_loader)

    for epoch in range(1, 1+config.EPOCHS):
        train(train_loader, model, optimizer, scheduler, device, epoch)
        valid(model, valid_loader, device, valid_df.caption.tolist()) 

        if epoch % 1 ==0:
            torch.save(model.state_dict(),os.path.join(f"model_test.pt"))

In [8]:
main()

NameError: name 'train_dataset' is not defined

# Model

In [152]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(config.ENCODER, config.DECODER)

(…)ase-patch16-224/resolve/main/config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.9.ln_cross_attn.weight', 'h.11.ln_cross_attn.weight', 'h.9.ln_cross_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.4.crossattention.c_proj.bias', 'h.5.crossattention.c_attn.bias', 'h.7.crossattention.q_attn.weight', 'h.9.crossattention.c_proj.weight', 'h.7.crossattention.c_attn.weight', 'h.6.crossattention.q_attn.bias', 'h.8.crossattention.q_attn.bias', 'h.3.crossattention.c_attn.weight', 'h.8.crossattention.c_proj.bias', 'h.6.crossattention.c_attn.weight', 'h.9.crossattention.q_attn.weight', 'h.6.crossattention.c_proj.weight', 'h.2.crossattention.q_attn.bias', 'h.3.crossattention.c_attn.bias', 'h.4.crossattention.q_attn.weight', 'h.7.crossattention.c_proj.weight', 'h.2.ln_cross_attn.weight', 'h.11.ln_cross_attn.bias', 'h.4.crossattention.c_proj.weight', 'h.0.crossattention.c_proj.bias', 'h.9.crossattention.q_attn.bias', 'h.4.ln_cross_attn.bias', 'h.8.ln_cross_attn

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [153]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size
# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

# Train

## Classic

In [159]:
# pip install transformers[torch]

zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [157]:
# !pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 615.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [160]:
training_args = Seq2SeqTrainingArguments(
    output_dir='VIT_large_gpt2',
    per_device_train_batch_size=config.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=config.VAL_BATCH_SIZE,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=1024,  
    save_steps=2048, 
    warmup_steps=1024,  
    learning_rate = 5e-5,
    #max_steps=1500, # delete for full training
    num_train_epochs = config.EPOCHS, #TRAIN_EPOCHS
    overwrite_output_dir=True,
    save_total_limit=1,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## Seq2Seq

In [ ]:
trainer = Seq2SeqTrainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
)
trainer.train()